In [1]:
import sys
sys.path.append('./../src')

In [445]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import datetime
import time
import os
import random
import re
from bs4 import BeautifulSoup
import requests
import web_scraping as ws
from importlib import reload
from jikanpy import Jikan
from copy import deepcopy
from joblib import Parallel, delayed
from math import sqrt
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chromedriver = "/Applications/chromedriver" # Path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

%config InlineBackend.figure_format = 'svg'
%matplotlib inline
sns.set(color_codes=True)
plt.style.use('seaborn-colorblind')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.precision', 2)

In [117]:
reload(ws)

<module 'web_scraping' from './../src/web_scraping.py'>

## Pickling ##

In [58]:
with open('../pickles/mal_ids_top_1000_anime.pkl', 'wb') as to_write:
    pickle.dump(mal_ids_top_1000_anime, to_write)

In [91]:
with open('../pickles/mal_ids_top_1000_anime.pkl', 'rb') as read_file:
    mal_ids_top_1000_anime = pickle.load(read_file)

In [82]:
with open('../pickles/mal_user_ids_120000.pkl', 'wb') as to_write:
    pickle.dump(mal_user_ids_120000, to_write)

In [438]:
with open('../pickles/top_anime_data_1000.pkl', 'wb') as to_write:
    pickle.dump(top_anime_data_1000, to_write)

In [443]:
with open('../pickles/top_anime_data_1000_df.pkl', 'wb') as to_write:
    pickle.dump(top_anime_data_1000_df, to_write)

In [437]:
top_anime_data_1000 = (top_anime_data_200
                       + top_anime_data_200_500
                       + top_anime_data_500_750
                       + top_anime_data_750_900
                       + top_anime_data_900_1000)

In [439]:
top_anime_data_1000_df = pd.DataFrame(top_anime_data_1000)

In [432]:
anime_data_top_200

[{'mal_id': 5114,
  'url': 'https://myanimelist.net/anime/5114',
  'image_url': 'https://cdn.myanimelist.net/images/anime/1223/96541.jpg',
  'trailer_url': 'https://www.youtube.com/embed/--IcmZkvL0Q?enablejsapi=1&wmode=opaque&autoplay=1',
  'title_main': 'Fullmetal Alchemist: Brotherhood',
  'title_english': 'Fullmetal Alchemist: Brotherhood',
  'media_type': 'TV',
  'source_material': 'Manga',
  'num_episodes': '64',
  'airing_status': 'Finished Airing',
  'aired_dates': 'Apr 5, 2009 to Jul 4, 2010',
  'premiered': 'Spring 2009',
  'duration': '24 min. per ep.',
  'content_rating': 'R - 17+ (violence & profanity)',
  'genres': ['Action',
   'Military',
   'Adventure',
   'Comedy',
   'Drama',
   'Magic',
   'Fantasy',
   'Shounen'],
  'score': '9.22',
  'scored_by_num_users': '1174592',
  'rank_score': '#1',
  'rank_popularity': '#4',
  'members': '1,926,949',
  'favorites': '161,402',
  'studios': ['Bones'],
  'producers': ['Aniplex',
   'Square Enix',
   'Mainichi Broadcasting Syste

## Playing with Jikan API ##

In [89]:
jikan = Jikan()

In [40]:
cowboy_bebop = jikan.anime(1)

In [62]:
type(jikan.anime(9253))

dict

In [41]:
cowboy_bebop

{'request_hash': 'request:anime:d6092f2422f084452c84555f17c7ba047e6998d3',
 'request_cached': True,
 'request_cache_expiry': -1598495124,
 'mal_id': 1,
 'url': 'https://myanimelist.net/anime/1/Cowboy_Bebop',
 'image_url': 'https://cdn.myanimelist.net/images/anime/4/19644.jpg',
 'trailer_url': 'https://www.youtube.com/embed/qig4KOK2R2g?enablejsapi=1&wmode=opaque&autoplay=1',
 'title': 'Cowboy Bebop',
 'title_english': 'Cowboy Bebop',
 'title_japanese': 'カウボーイビバップ',
 'title_synonyms': [],
 'type': 'TV',
 'source': 'Original',
 'episodes': 26,
 'status': 'Finished Airing',
 'airing': False,
 'aired': {'from': '1998-04-03T00:00:00+00:00',
  'to': '1999-04-24T00:00:00+00:00',
  'prop': {'from': {'day': 3, 'month': 4, 'year': 1998},
   'to': {'day': 24, 'month': 4, 'year': 1999}},
  'string': 'Apr 3, 1998 to Apr 24, 1999'},
 'duration': '24 min per ep',
 'rating': 'R - 17+ (violence & profanity)',
 'score': 8.79,
 'scored_by': 527450,
 'rank': 27,
 'popularity': 36,
 'members': 1064382,
 'fa

In [14]:
killuagon646 = jikan.user(username='killuagon646', request='animelist', argument='all')

In [15]:
killuagon646

{'request_hash': 'request:user:e72bba7eb603e9a01f8eec7acc5860b847463cd5',
 'request_cached': False,
 'request_cache_expiry': 3600,
 'anime': [{'mal_id': 38883,
   'title': 'Haikyuu!!: To the Top',
   'video_url': 'https://myanimelist.net/anime/38883/Haikyuu__To_the_Top/video',
   'url': 'https://myanimelist.net/anime/38883/Haikyuu__To_the_Top',
   'image_url': 'https://cdn.myanimelist.net/images/anime/1813/105367.jpg?s=af27994870941c71f53cd94e3fa0fa89',
   'type': 'TV',
   'watching_status': 1,
   'score': 9,
   'watched_episodes': 13,
   'total_episodes': 13,
   'airing_status': 2,
   'season_name': None,
   'season_year': None,
   'has_episode_video': False,
   'has_promo_video': True,
   'has_video': True,
   'is_rewatching': False,
   'tags': None,
   'rating': 'PG-13',
   'start_date': '2020-11-01T00:00:00+00:00',
   'end_date': '2020-04-04T00:00:00+00:00',
   'watch_start_date': None,
   'watch_end_date': None,
   'days': None,
   'storage': None,
   'priority': 'Low',
   'added_

In [56]:
jikan.top(type='anime', page=1)

{'request_hash': 'request:top:58399c95e55435d6ccef63eef7ce609974e4f2d5',
 'request_cached': True,
 'request_cache_expiry': 228985,
 'top': [{'mal_id': 5114,
   'rank': 1,
   'title': 'Fullmetal Alchemist: Brotherhood',
   'url': 'https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood',
   'image_url': 'https://cdn.myanimelist.net/images/anime/1223/96541.jpg?s=faffcb677a5eacd17bf761edd78bfb3f',
   'type': 'TV',
   'episodes': 64,
   'start_date': 'Apr 2009',
   'end_date': 'Jul 2010',
   'members': 1916739,
   'score': 9.22},
  {'mal_id': 9253,
   'rank': 2,
   'title': 'Steins;Gate',
   'url': 'https://myanimelist.net/anime/9253/Steins_Gate',
   'image_url': 'https://cdn.myanimelist.net/images/anime/5/73199.jpg?s=97b97d568f25a02cf5a22dda13b5371f',
   'type': 'TV',
   'episodes': 24,
   'start_date': 'Apr 2011',
   'end_date': 'Sep 2011',
   'members': 1556283,
   'score': 9.13},
  {'mal_id': 11061,
   'rank': 3,
   'title': 'Hunter x Hunter (2011)',
   'url': 'https://myan

In [10]:
top_anime_page1['top']

[{'mal_id': 5114,
  'rank': 1,
  'title': 'Fullmetal Alchemist: Brotherhood',
  'url': 'https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood',
  'image_url': 'https://cdn.myanimelist.net/images/anime/1223/96541.jpg?s=faffcb677a5eacd17bf761edd78bfb3f',
  'type': 'TV',
  'episodes': 64,
  'start_date': 'Apr 2009',
  'end_date': 'Jul 2010',
  'members': 1916739,
  'score': 9.22},
 {'mal_id': 9253,
  'rank': 2,
  'title': 'Steins;Gate',
  'url': 'https://myanimelist.net/anime/9253/Steins_Gate',
  'image_url': 'https://cdn.myanimelist.net/images/anime/5/73199.jpg?s=97b97d568f25a02cf5a22dda13b5371f',
  'type': 'TV',
  'episodes': 24,
  'start_date': 'Apr 2011',
  'end_date': 'Sep 2011',
  'members': 1556283,
  'score': 9.13},
 {'mal_id': 11061,
  'rank': 3,
  'title': 'Hunter x Hunter (2011)',
  'url': 'https://myanimelist.net/anime/11061/Hunter_x_Hunter_2011',
  'image_url': 'https://cdn.myanimelist.net/images/anime/11/33657.jpg?s=5724d8c22ae7a1dad72d8f4229ef803f',
  'type': 

In [87]:
mal_ids_top_1000_anime = get_top_anime_mal_ids(1000)

NameError: name 'get_top_anime_mal_ids' is not defined

In [104]:
anime_data_top_50[-1]

{'request_hash': 'request:anime:13e4850dd1d42438d8442ebe82e5f4ded43ec08c',
 'request_cached': True,
 'request_cache_expiry': 311136,
 'mal_id': 37779,
 'url': 'https://myanimelist.net/anime/37779/Yakusoku_no_Neverland',
 'image_url': 'https://cdn.myanimelist.net/images/anime/1125/96929.jpg',
 'trailer_url': 'https://www.youtube.com/embed/JIcjo7XVlOY?enablejsapi=1&wmode=opaque&autoplay=1',
 'title': 'Yakusoku no Neverland',
 'title_english': 'The Promised Neverland',
 'title_japanese': '約束のネバーランド',
 'title_synonyms': [],
 'type': 'TV',
 'source': 'Manga',
 'episodes': 12,
 'status': 'Finished Airing',
 'airing': False,
 'aired': {'from': '2019-01-10T00:00:00+00:00',
  'to': '2019-03-29T00:00:00+00:00',
  'prop': {'from': {'day': 10, 'month': 1, 'year': 2019},
   'to': {'day': 29, 'month': 3, 'year': 2019}},
  'string': 'Jan 10, 2019 to Mar 29, 2019'},
 'duration': '22 min per ep',
 'rating': 'R - 17+ (violence & profanity)',
 'score': 8.66,
 'scored_by': 452576,
 'rank': 50,
 'popularit

In [102]:
anime_data_top_50 = deepcopy(anime_data_top_100)

In [98]:
anime_data_top_50 = get_anime_data(mal_ids_top_1000_anime[:50])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [71]:
anime_data_test[2]

{'request_hash': 'request:anime:e09f711f9e532dc4ffb4d5649b5dbd6e0fb25648',
 'request_cached': True,
 'request_cache_expiry': 311543,
 'mal_id': 11061,
 'url': 'https://myanimelist.net/anime/11061/Hunter_x_Hunter_2011',
 'image_url': 'https://cdn.myanimelist.net/images/anime/11/33657.jpg',
 'trailer_url': 'https://www.youtube.com/embed/D9iTQRB4XRk?enablejsapi=1&wmode=opaque&autoplay=1',
 'title': 'Hunter x Hunter (2011)',
 'title_english': 'Hunter x Hunter',
 'title_japanese': 'HUNTER×HUNTER（ハンター×ハンター）',
 'title_synonyms': ['HxH (2011)'],
 'type': 'TV',
 'source': 'Manga',
 'episodes': 148,
 'status': 'Finished Airing',
 'airing': False,
 'aired': {'from': '2011-10-02T00:00:00+00:00',
  'to': '2014-09-24T00:00:00+00:00',
  'prop': {'from': {'day': 2, 'month': 10, 'year': 2011},
   'to': {'day': 24, 'month': 9, 'year': 2014}},
  'string': 'Oct 2, 2011 to Sep 24, 2014'},
 'duration': '23 min per ep',
 'rating': 'PG-13 - Teens 13 or older',
 'score': 9.12,
 'scored_by': 764111,
 'rank': 3,

## Scraping ##

In [61]:
mal_users_url = 'https://myanimelist.net/users.php?cat=user&q=&loc=&agelow=18&agehigh=80&g='

In [111]:
BASE_URL = 'https://myanimelist.net/anime/'

In [230]:
soup = ws.create_soup('https://myanimelist.net/anime/40956/')

In [415]:
reload(ws)

<module 'web_scraping' from './../src/web_scraping.py'>

In [286]:
ws.get_licensors(40956, BASE_URL)
ws.get_licensors(1, BASE_URL)

['Funimation']

['Funimation', 'Bandai Entertainment']

In [433]:
top_anime_data_200 = Parallel(n_jobs=-1, verbose=5)(map(delayed(ws.get_anime_data), mal_ids_top_1000_anime[:200]))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   17.4s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   41.1s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:   56.7s finished


In [446]:
driver = webdriver.Chrome(chromedriver)
driver.get('https://myanimelist.net/animelist/killuagon646')

In [448]:
soup = BeautifulSoup(driver.page_source, 'html5lib')

In [452]:
# Index
soup.find('tbody', class_='list-item').find(class_='data number').text

'1'

In [449]:
# Title
soup.find('tbody', class_='list-item').find(class_='data title clearfix').find(class_='link sort').text

'Enen no Shouboutai: Ni no Shou'

In [450]:
# Score
soup.find('tbody', class_='list-item').find(class_='data score').text.strip()

'7'